<a href="https://www.kaggle.com/code/cedricsvrn/h-m-eda-baseline?scriptVersionId=99783665" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd

from collections import Counter

transactions = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
sample_submission = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
# /kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv
# /kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv
# /kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv
# /kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv

In [2]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [3]:
sample_submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


# Cleaning + create train and test dataset

In [4]:
transactions['article_id'] = transactions['article_id'].astype(str)

In [5]:
test_set = 0.3
train_set = (1-test_set)
split = int(transactions.shape[0]*train_set)

transactions_train = transactions.sort_values('t_dat').iloc[:split]
transactions_test = transactions.sort_values('t_dat').iloc[split:]
print("Train set size :", transactions_train.shape[0])
print("Test set size :", transactions_test.shape[0])

Train set size : 22251826
Test set size : 9536498


# Best sellers recommender

In [6]:
class BestSellersRecsys():
    """FIXME"""
    def __init__(self):
        """FIXME"""
        pass
    
    def fit(self, X):
        """FIXME"""
        self.X_occur = Counter(list(X['article_id']))
        self.best_sellers_10 = [article_id for (article_id, occ) in self.X_occur.most_common(10)]
        return self
    
    def predict(self, basket, k=10):
        """FIXME"""
        if k==10:
            return self.best_sellers_10
        else:
            best_sellers = [article_id for (article_id, occ) in self.X_occur.most_common(k)]
            return best_sellers

In [7]:
bs_recsys = BestSellersRecsys()
bs_recsys.fit(transactions_train)

In [8]:
bs_recsys.predict([])

['706016001',
 '706016002',
 '372860001',
 '399223001',
 '562245001',
 '156231001',
 '464297007',
 '610776002',
 '673677002',
 '568601006']

# CBF : Cosine Similarity

In [9]:
from math import sqrt

class CosineSimCoOccurenceRecsys():
    """FIXME"""
    def __init__(self):
        """FIXME"""
        pass
    
    def _get_l_of_two_from_l(self, l):
        """FIXME"""
        if len(l)>1:
            l_of_two = []
            for i in range(0,len(l)):
                for j in range(i+1,len(l)):
                    if int(l[i])<int(l[j]):
                        l_of_two.append(f"{l[i]}|{l[j]}")
                    else:
                        l_of_two.append(f"{l[j]}|{l[i]}")
            return l_of_two
        return None
    
    def _get_frequency(self, article_id, counter_occur_article_id):
        """FIXME"""
        return counter_occur_article_id[article_id]
    
    def _get_co_occ_frequency(self, article_id_a, article_id_b, counter_co_occ_article_id):
        """FIXME"""
        if int (article_id_a) < int(article_id_b):
            str_co_occ = article_id_a + '|' + article_id_b
        else:
            str_co_occ = article_id_b + '|' + article_id_a
        return counter_co_occ_article_id[str_co_occ]
    
    def _get_cosine_sim(self, freq_a, freq_b, co_occ_freq):
        """FIXME"""
        return (co_occ_freq/sqrt(freq_a*freq_b))
        
    def fit(self, X):
        """FIXME"""        
        # Getting frequencies of article_ids
        counter_occur_article_id = Counter(list(X['article_id']))
        
        # Getting baskets
        # FIXME 
        l_baskets = list(X.head(1000000).groupby(['t_dat', 'customer_id'])['article_id'].apply(list).reset_index()['article_id'])

        # Getting co-occur frequencies of article_ids
        l_co_occur_article_id = []
        for basket in l_baskets:
            l_of_two = self._get_l_of_two_from_l(basket)
            if l_of_two:
                l_co_occur_article_id += l_of_two

        counter_co_occ_article_id = Counter(l_co_occur_article_id)
        
        # Creating pandas datframe
        l_article_id_a = [i.split('|', 1)[0] for i in l_co_occur_article_id]
        l_article_id_b = [i.split('|', 1)[1] for i in l_co_occur_article_id]
        df_cosine = pd.DataFrame()
        df_cosine['article_id_a'] = l_article_id_a
        df_cosine['article_id_b'] = l_article_id_b
        df_cosine['freq_a'] = df_cosine['article_id_a'].apply(lambda x : self._get_frequency(x, counter_occur_article_id))
        df_cosine['freq_b'] = df_cosine['article_id_b'].apply(lambda x : self._get_frequency(x, counter_occur_article_id))
        df_cosine['co_occ_freq'] = df_cosine.apply(lambda x: self._get_co_occ_frequency(x.article_id_a, x.article_id_b, counter_co_occ_article_id), axis=1)
        df_cosine['cosine_sim'] = df_cosine.apply(lambda x: self._get_cosine_sim(x.freq_a, x.freq_b, x.co_occ_freq), axis=1)
        df_cosine = df_cosine.sort_values('cosine_sim', ascending=False)
        self.df_cosine = df_cosine
        return self
    
    def predict(self, basket, k=10, strat='mean'):
        """FIXME"""
        df = self.df_cosine[(self.df_cosine['article_id_a'].isin(basket)) & (~self.df_cosine['article_id_b'].isin(basket))]
        if strat=='mean':
            result = list(df[['article_id_b', 'cosine_sim']].groupby(['article_id_b']).mean().reset_index()['article_id_b'])[:k]
        elif strat=='max':
            result = list(df[['article_id_b', 'cosine_sim']].groupby(['article_id_b']).max().reset_index()['article_id_b'])[:k]
        else:
            rsult = []
        return result

In [10]:
cs_recsys = CosineSimCoOccurenceRecsys().fit(transactions_train)

In [11]:
cs_recsys.predict(['541518023'])

['541519001',
 '541519005',
 '541519024',
 '541519028',
 '543141003',
 '543310001',
 '543669001',
 '546661001',
 '547333002',
 '547345003']

# Evaluating recsys

In [12]:
import random

def is_predicted(customer_id, order, model, k):
    """FIXME"""
    order_ = order.copy()
    hidden_article_id = order_.pop(random.randrange(len(order_)))
    visible_order = order_

    predictions = model.predict(visible_order, k)
    if hidden_article_id in predictions:
        return 1
    else:
        return 0

In [13]:
def history_basket_performance(X, model, k=10):
    """FIXME"""
    # Grouping to have orders by day
    X_orders = X.groupby(['t_dat', 'customer_id'])['article_id'].apply(list).reset_index()    
    
    # Keeping only orders where we can hide at least one article and have at least one item in our basket
    X_orders['is_long_enough'] = X_orders['article_id'].apply(lambda x: 1 if(len(x)>1) else 0)
    X_orders = X_orders[X_orders['is_long_enough']==1]
    X_orders = X_orders.drop('is_long_enough', 1)
    
    X_orders['predicted'] = X_orders.apply(lambda x: is_predicted(x.customer_id, x.article_id, model, k), axis=1)
    is_predicted_count = X_orders[X_orders['predicted']==1].shape[0]
    count = X_orders.shape[0]
    
    return (count, is_predicted_count/count)

In [14]:
history_basket_performance(transactions_test.head(1000), bs_recsys)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if __name__ == '__main__':


(233, 0.012875536480686695)

In [15]:
history_basket_performance(transactions_test.head(1000), cs_recsys)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  if __name__ == '__main__':


(233, 0.004291845493562232)

In [16]:
# (205, 0.24878048780487805)